In [1]:
import numpy as np
import hyperspy.api as hs
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5


### Loading data

In [4]:
s = hs.load("680.rpl").as_signal1D(0)


In [6]:
s.plot()

In [7]:
s.axes_manager.gui()

### Loading parameters

In [8]:
s.set_signal_type("EDS_SEM")

In [9]:
s.axes_manager[-1].name = 'E'
s.axes_manager['E'].units = 'keV'
s.axes_manager['E'].scale = 0.01
s.axes_manager['E'].offset = -0.1

In [10]:
s.add_elements(['Ge','Al','P','O'])

In [11]:
s.set_microscope_parameters(beam_energy=5.0)

In [12]:
s.add_lines(['O_Ka','Al_Ka','P_Ka','Ge_La'])

In [13]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 35.0
│       │       └── energy_resolution_MnKa = 130.0
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       └── beam_energy = 5.0
├── General
│   ├── date = 
│   ├── original_filename = 680.rpl
│   ├── time = 
│   └── title = 
├── Sample
│   ├── elements = ['Al', 'Ge', 'O', 'P']
│   └── xray_lines = ['Al_Ka', 'Ge_La', 'O_Ka', 'P_Ka']
└── Signal
    ├── binned = True
    └── signal_type = EDS_SEM

In [14]:
s.crop_signal1D(0.3, 2.5)

In [15]:
s.plot()

In [16]:
s.plot(xray_lines=True)

In [17]:
s.change_dtype('float')

Traceback (most recent call last):
  File "C:\Users\UPJV\HyperSpy-bundle\lib\site-packages\matplotlib\cbook\__init__.py", line 196, in process
    func(*args, **kwargs)
  File "C:\Users\UPJV\HyperSpy-bundle\lib\site-packages\hyperspy\drawing\utils.py", line 192, in function_wrapper
    function()
  File "C:\Users\UPJV\HyperSpy-bundle\lib\site-packages\hyperspy\drawing\signal1d.py", line 145, in _on_close
    super(Signal1DFigure, self)._on_close()
  File "C:\Users\UPJV\HyperSpy-bundle\lib\site-packages\hyperspy\drawing\figure.py", line 117, in _on_close
    self.events.closed.trigger(obj=self)
  File "<string>", line 4, in trigger
  File "C:\Users\UPJV\HyperSpy-bundle\lib\site-packages\hyperspy\events.py", line 422, in trigger
    function(**{kw: kwargs.get(kw, None) for kw in kwsl})
  File "C:\Users\UPJV\HyperSpy-bundle\lib\site-packages\hyperspy\signal.py", line 2672, in <lambda>
    lambda: self.events.data_changed.disconnect(self.update_plot),
  File "C:\Users\UPJV\HyperSpy-bundle\

In [18]:
s.save('spectrum.hdf5')

Overwrite 'spectrum.hdf5' (y/n)?
y


### Loading data

In [44]:
sem_im = hs.load('680segmented2048.tif')

In [45]:
sem_im.plot()

In [46]:
s = hs.load('spectrum.hdf5')

In [47]:
s.data.shape

(1536, 2048, 220)

In [48]:
sem_im.data.shape

(1536, 2048)

In [ ]:
# Look for the pixel values in the SEM images
max_px_nbr = 3 # input the maximal number of different areas expected
px_values = np.zeros((max_px_nbr), dtype='int')

s_single_sum = hs.hyperspy.signal.hyperspy.signals.EDSSEMSpectrum( \
                    np.zeros([max_px_nbr,s.data.shape[2]], dtype='float'))

k_0=0
k_1=0
K_2=0
s_single_sum.data[:]=0

for i in range(0,sem_im.data.shape[0]):
    for j in range(0, sem_im.data.shape[1]):
        for k in range(0, px_values.size):
            if sem_im.isig[j,i]()[0]==0:
                s_single_sum.inav[0]=s_single_sum.inav[0]+s.inav[j,i]
                k_0 = k_0 +1
            elif sem_im.isig[j,i]()[0]==1:
                s_single_sum.inav[1]=s_single_sum.inav[1]+s.inav[j,i]
                k_1 = k_1 +1
            else: print('pb')

In [27]:
s_single_sum2 = s_single_sum.deepcopy()

In [28]:
s_single_sum.add_elements(['Ge','Al','P','O'])

In [29]:
s_single_sum.axes_manager[1].offset = s.axes_manager[2].offset 
s_single_sum.axes_manager[1].scale = s.axes_manager[2].scale 
s_single_sum.axes_manager[1].units = s.axes_manager[2].units

In [30]:
s_single_sum.add_lines(['Al_Ka','P_Ka','Ge_La','O_Ka'])

In [31]:
s_single_sum.metadata.Acquisition_instrument = s.metadata.Acquisition_instrument
s_single_sum.metadata.Sample = s.metadata.Sample

In [32]:
s_single_sum.plot(xray_lines=True)

In [33]:
s_single_sum.save('Sum2.hdf5')

In [34]:
s_single_sum = hs.load('Sum2.hdf5')

In [35]:
s.plot()

In [36]:
s_single_sum.plot()

In [37]:
s.add_lines(['Al_Ka','P_Ka','Ge_La','O_Ka'])

In [38]:
bw = s_single_sum.estimate_background_windows(line_width=[2.5, 2.0])
s_single_sum.plot(background_windows=bw)
intensities = s_single_sum.get_lines_intensity(background_windows=bw)

In [39]:
intensities

[<BaseSignal, title: X-ray line intensity of : Al_Ka at 1.49 keV, dimensions: (3|)>,
 <BaseSignal, title: X-ray line intensity of : Ge_La at 1.19 keV, dimensions: (3|)>,
 <BaseSignal, title: X-ray line intensity of : O_Ka at 0.52 keV, dimensions: (3|)>,
 <BaseSignal, title: X-ray line intensity of : P_Ka at 2.01 keV, dimensions: (3|)>]

In [40]:
# intensities are the intensities- 0: Al_Ka, 1: Ge_La, 2:O_Ka, 3:P_Ka 
# inav indicates which area to selec-t 0: inter-grain, 1:grain, 2:voids
# intensities are the X-rays intensities, they do not directly give the concentration
# To get the concentration the .quantification method should be used by needs the CL factors \
#       from the X-rays acquisition software

In [41]:
# Al/Ge inter-grain
intensities[0].inav[0].data[0]/intensities[1].inav[0].data[0]

<ipython-input-41-2e1cfb8fe675>:2: RuntimeWarning: invalid value encountered in double_scalars
  intensities[0].inav[0].data[0]/intensities[1].inav[0].data[0]


nan

In [42]:
# Al/Ge grain
intensities[0].inav[1].data[0]/intensities[1].inav[1].data[0]

0.008564902931100115

In [43]:
# O/Ge inter-grain
intensities[2].inav[0].data[0]/intensities[1].inav[0].data[0]

<ipython-input-43-d21d11f089f2>:2: RuntimeWarning: invalid value encountered in double_scalars
  intensities[2].inav[0].data[0]/intensities[1].inav[0].data[0]


nan

In [30]:
# O/Ge grain
intensities[2].inav[1].data[0]/intensities[1].inav[1].data[0]

2.1970664880903867

In [31]:
# P/Ge inter-grain
intensities[3].inav[0].data[0]/intensities[1].inav[0].data[0]

1.286965674124877

In [32]:
# P/Ge grain
intensities[3].inav[1].data[0]/intensities[1].inav[1].data[0]

0.6182579560799276

In [46]:
#O/Al intergrain
intensities[2].inav[0].data[0]/intensities[0].inav[0].data[0]

25.504716981132077

In [47]:
# O/Al grain
intensities[2].inav[1].data[0]/intensities[0].inav[1].data[0]

19.068588089092824

In [48]:
#P/Al intergrain
intensities[3].inav[0].data[0]/intensities[0].inav[0].data[0]

9.535849056603773

In [49]:
#P/Al grain
intensities[3].inav[1].data[0]/intensities[0].inav[1].data[0]

5.365930599369086

In [50]:
#O/P intergrain
intensities[2].inav[0].data[0]/intensities[3].inav[0].data[0]

2.6746141669964385

In [52]:
#O/P grain
intensities[2].inav[1].data[0]/intensities[3].inav[1].data[0]

3.553640461047869

In [53]:
s_single_sum.plot(xray_lines=True)

In [54]:
s_single_sum.inav[0].plot()
###spectra of intergrain region

In [55]:
s_single_sum.inav[1].plot()
###Spectra of the grain region

In [59]:
all_spectra= hs.plot.plot_spectra([s_single_sum.inav[0], s_single_sum.inav[1]])
### blue, yellow

In [57]:
s_single_sum.axes_manager[-1].name = 'E'
s_single_sum.axes_manager['E'].units = 'keV'

In [60]:
all_spectra.figure.savefig("all_spectra_non_normalized.png")

In [61]:
### NORMALIZATION
for i in s_single_sum:
    i.data /= i.data.max()

In [62]:
norma_all_spectra= hs.plot.plot_spectra([s_single_sum.inav[0], s_single_sum.inav[1]])
plt.draw()

In [63]:
norma_all_spectra.figure.savefig("norm_spectra700.png")

In [65]:
#Ge grain/Ge inter grain
intensities[1].inav[1].data[0]/intensities[1].inav[0].data[0]

15.413064883033986

In [66]:
#Al grain/Al inter grain
intensities[0].inav[1].data[0]/intensities[0].inav[0].data[0]

13.158490566037736

In [67]:
#O grain/O inter grain
intensities[2].inav[1].data[0]/intensities[2].inav[0].data[0]

9.837938474816596

In [68]:
#P grain/P inter grain
intensities[3].inav[1].data[0]/intensities[3].inav[0].data[0]

7.404432132963989

# Example quantification
s = hs.datasets.example_signals.EDS_TEM_Spectrum()
s.add_lines()
kfactors = [1.450226, 5.075602] #For Fe Ka and Pt La
bw = s.estimate_background_windows(line_width=[5.0, 2.0])
intensities = s.get_lines_intensity(background_windows=bw)
atomic_percent = s.quantification(intensities, method='CL', factors=kfactors)
Fe (Fe_Ka): Composition = 15.41 atomic percent
Pt (Pt_La): Composition = 84.59 atomic percent